### Import des bibliothéques
---

In [16]:
import pandas as pd
import requests as rq
import json as js
import time


### Données du projet : Top 35 Villes
---

In [3]:
Top_ville = [
    "Mont Saint Michel", "St Malo", "Bayeux", "Le Havre", "Rouen",
    "Paris", "Amiens", "Lille", "Strasbourg", "Chateau du Haut Koenigsbourg",
    "Colmar", "Eguisheim", "Besancon", "Dijon", "Annecy",
    "Grenoble", "Lyon", "Gorges du Verdon", "Bormes les Mimosas", "Cassis",
    "Marseille", "Aix en Provence", "Avignon", "Uzes", "Nimes",
    "Aigues Mortes", "Saintes Maries de la Mer", "Collioure",
    "Carcassonne", "Ariege", "Toulouse", "Montauban", "Biarritz",
    "Bayonne", "La Rochelle"
]


### Scraping des coordonnées GPS
---

In [12]:
URL = 'https://nominatim.openstreetmap.org/search'
headers = {'User-Agent': 'MonProjetKayak/1.0'}
ville = "St Malo"
params = {'q': f"{ville}, France", 'format': 'json'}

# Test sur St Malo
response=rq.get(
    URL,
    params=params,
    headers=headers
    )

response.json()


[{'place_id': 262173744,
  'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. http://osm.org/copyright',
  'osm_type': 'relation',
  'osm_id': 905534,
  'lat': '48.6495180',
  'lon': '-2.0260409',
  'class': 'boundary',
  'type': 'administrative',
  'place_rank': 16,
  'importance': 0.6170333686756257,
  'addresstype': 'town',
  'name': 'Saint-Malo',
  'display_name': 'Saint-Malo, Ille-et-Vilaine, Bretagne, France métropolitaine, 35400, France',
  'boundingbox': ['48.5979853', '48.6949736', '-2.0765246', '-1.9367259']},
 {'place_id': 261847659,
  'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. http://osm.org/copyright',
  'osm_type': 'relation',
  'osm_id': 1653637,
  'lat': '48.4904728',
  'lon': '-1.7421555',
  'class': 'boundary',
  'type': 'administrative',
  'place_rank': 14,
  'importance': 0.5038359135553251,
  'addresstype': 'municipality',
  'name': 'Saint-Malo',
  'display_name': 'Saint-Malo, Ille-et-Vilaine, Bretagne, France métropolitaine, France',
  'boundin

In [18]:
# Liste temporaire pour stocker les lignes
rows = []

for ville in Top_ville:
    try:
        params = {'q': f"{ville}, France", 'format': 'json', 'limit': 1}
        response = rq.get(URL, params=params, headers=headers, timeout=10)
        response.raise_for_status()
        data = response.json()

        if data:
            lat = float(data[0]['lat'])
            lon = float(data[0]['lon'])
            rows.append([ville, lat, lon])
            print(f"{ville} -> OK ({lat}, {lon})")
        else:
            print(f"\tAucune donnée trouvée pour {ville}")

    except rq.RequestException as e:
        print(f"\tErreur pour {ville} : {e}")

    time.sleep(1)  # pour ne pas saturer l'API

# Création finale du DataFrame
df_ville = pd.DataFrame(rows, columns=['nom_ville','latitude','longitude'])

df_ville.head()

Mont Saint Michel -> OK (48.6359541, -1.51146)
St Malo -> OK (48.649518, -2.0260409)
Bayeux -> OK (49.2764624, -0.7024738)
Le Havre -> OK (49.4938975, 0.1079732)
Rouen -> OK (49.4404591, 1.0939658)
Paris -> OK (48.8534951, 2.3483915)
Amiens -> OK (49.8941708, 2.2956951)
Lille -> OK (50.6365654, 3.0635282)
Strasbourg -> OK (48.584614, 7.7507127)
Chateau du Haut Koenigsbourg -> OK (48.2494107, 7.3443202)
Colmar -> OK (48.0777517, 7.3579641)
Eguisheim -> OK (48.0447968, 7.3079618)
Besancon -> OK (47.2380222, 6.0243622)
Dijon -> OK (47.3215806, 5.0414701)
Annecy -> OK (45.8992348, 6.1288847)
Grenoble -> OK (45.1875602, 5.7357819)
Lyon -> OK (45.7578137, 4.8320114)
Gorges du Verdon -> OK (43.7496562, 6.3285616)
Bormes les Mimosas -> OK (43.1506968, 6.3419285)
Cassis -> OK (43.2140359, 5.5396318)
Marseille -> OK (43.2961743, 5.3699525)
Aix en Provence -> OK (43.5298424, 5.4474738)
Avignon -> OK (43.9492493, 4.8059012)
Uzes -> OK (44.0121279, 4.4196718)
Nimes -> OK (43.8374249, 4.3600687)
Aig

,nom_ville,latitude,longitude
0,Mont Saint Michel,48.635954,-1.511460
1,St Malo,48.649518,-2.026041
2,Bayeux,49.276462,-0.702474
3,Le Havre,49.493898,0.107973
4,Rouen,49.440459,1.093966


### Création de la colonne id_ville pour les futurs tables SQL
---

In [19]:
df_ville['id_ville']=range(1, len(df_ville)+1)
df_ville.head()


,nom_ville,latitude,longitude,id_ville
0,Mont Saint Michel,48.635954,-1.511460,1
1,St Malo,48.649518,-2.026041,2
2,Bayeux,49.276462,-0.702474,3
3,Le Havre,49.493898,0.107973,4
4,Rouen,49.440459,1.093966,5


### Sauvegarde df_ville
---

In [20]:
df_ville.to_csv('df_ville.csv', index=False)